In [3]:
import pandas as pd
import numpy as np
import codecs
import re
from os import path, listdir

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [8]:
# clone repo
# %cd '/content/drive/My Drive/Colab Notebooks/tpa/'
%rm -rf repo/
%rm -rf frags/*
!git clone https://github.com/r0mflip/syn-data ./repo
!find repo/data/ -type f -name '*.csv' -exec cp -f {} ./frags/ \; && echo 'Done copying data files'

Cloning into './repo'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 379 (delta 0), reused 4 (delta 0), pack-reused 374
Receiving objects: 100% (379/379), 79.03 MiB | 5.02 MiB/s, done.
Resolving deltas: 100% (100/100), done.
Updating files: 100% (333/333), done.
Done copying data files


In [9]:
# pwd = '/content/drive/My Drive/Colab Notebooks/tpa/'
pwd = './'
frags_dir = path.join(pwd, 'frags')
monolith_dir = path.join(pwd, 'monolith')

In [10]:
# Get all csv files
src_files = [path.join(frags_dir, f) for f in listdir(frags_dir) if path.isfile(path.join(frags_dir, f)) and f.endswith('.csv')]
print(len(src_files), src_files)
# src_files = ['/content/drive/My Drive/Colab Notebooks/tpa/frags/1582563810verybigcsvah.csv']

310 ['./frags/1582563810verybigcsvah.csv', './frags/1582563810verybigcsvig.csv', './frags/1582563810verybigcsvcb.csv', './frags/1582563810verybigcsvhg.csv', './frags/1582563810verybigcsvfa.csv', './frags/1582563810verybigcsvdl.csv', './frags/1582563810verybigcsvhp.csv', './frags/1582563810verybigcsvby.csv', './frags/1582563810verybigcsvju.csv', './frags/1582563810verybigcsvke.csv', './frags/1582563810verybigcsvai.csv', './frags/1582563810verybigcsvkl.csv', './frags/1582563810verybigcsviz.csv', './frags/1582563810verybigcsvix.csv', './frags/1582563810verybigcsvhv.csv', './frags/1582563810verybigcsvee.csv', './frags/1582563810verybigcsvif.csv', './frags/1582563810verybigcsvcz.csv', './frags/1582563810verybigcsvgc.csv', './frags/1582563810verybigcsvgo.csv', './frags/1582563810verybigcsvdz.csv', './frags/1582563810verybigcsvck.csv', './frags/1582563810verybigcsvcy.csv', './frags/1582563810verybigcsvao.csv', './frags/1582563810verybigcsvbv.csv', './frags/1582563810verybigcsvid.csv', './frag

In [11]:
def defc(s):
  '''
  double decode text out of escape, remove 'byte marking'
  and newlines using regex, strip text and return

  should print out the dirty text on error
  '''
  try:
    return (
      re.search(
        r'^b[\'|\"]+\'*\"*(.*)\'*\"*[\'|\"]+$',
        codecs.decode(
          codecs.decode(
            s,
            'unicode-escape'
          ), 'unicode-escape'
        ).replace('\n', '')
      ).group(1)
    ).strip()
  except Exception as e:
    print(s, e)
    return ''

In [ ]:
# save all unique tweets into a single file
cols = ['tweet_text', 'tweet_id', 'user_name', 'user_screen_name',
        'user_location', 'user_description', 'user_follower_count',
        'user_friends_count', 'user_listed_count', 'user_statuses_count',
        'user_favourites_count', 'user_verified', 'user_default_profile_image',
        'user_default_profile', 'user_protected', 'user_created_at']

# root df
root = pd.DataFrame(columns=cols)

# dict of unique tweets
tweets = {}

# all files
for frag in src_files:
  df = pd.read_csv(frag, names=cols, header=0)
  # sanitize text
  dfc = df.applymap(defc)
  for _, row in dfc.iterrows():
    if str(row[['tweet_id']]) == 'tweet_id':
      break
    tweet_id = str(row[['tweet_id']])
    if tweet_id not in tweets:
      tweets[tweet_id] = row.to_list()

# print number of unique tweets
print('Unique tweets:', len(tweets))

# convert all into list of series and create a new df
tweet_items = [k for k in list(tweets.values())]

In [17]:
tweets_series = list(map(lambda row : pd.Series(row, index=root.columns), tweet_items))
root = root.append(tweets_series, ignore_index=True)
print(root.shape)


# root.fillna(0)
# save to csv file
# root.to_csv(path.join(monolith_dir, 'final_tweets.csv'), index=False)

(581762, 16)


In [39]:
# root.replace('', np.nan, inplace=True)
root.isna().any()

tweet_text                    False
tweet_id                      False
user_name                      True
user_screen_name              False
user_location                  True
user_description               True
user_follower_count           False
user_friends_count            False
user_listed_count             False
user_statuses_count           False
user_favourites_count         False
user_verified                 False
user_default_profile_image    False
user_default_profile          False
user_protected                False
user_created_at               False
dtype: bool

In [36]:
print(root.shape)
print(root.dtypes)
root.to_csv(path.join(monolith_dir, 'final_tweets.csv'), index=False)

(581762, 16)
tweet_text                    object
tweet_id                      object
user_name                     object
user_screen_name              object
user_location                 object
user_description              object
user_follower_count           object
user_friends_count            object
user_listed_count             object
user_statuses_count           object
user_favourites_count         object
user_verified                 object
user_default_profile_image    object
user_default_profile          object
user_protected                object
user_created_at               object
dtype: object
